In [7]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [1]:
def Conv2D(image, filters, stride, padding):
    h, w, c = image.shape
    f, f, n_of_filters = filters.shape
    output_images = []
    for i in range(n_of_filters):
        for j in range(c):
            if padding == 'same':
                p = (f - 1) // 2
                padded_image = np.pad(image[:,:,j], ((p, p), (p, p)), mode='constant')
                output_image = np.zeros((h, w))
            else:
                padded_image = image[:,:,j]
                output_image = np.zeros((h - f + 1, w - f + 1))

            for k in range(0, h, stride):
                for l in range(0, w, stride):
                    roi = padded_image[k:k+f, l:l+f]
                    output_image[k, l] += np.sum(roi * filters[:,:,i])

        output_images.append(output_image)

    subplot_count = len(output_images) + 1
    plt.subplot(subplot_count, 1)
    plt.imshow(image)
    for i in range(1, subplot_count):
        plt.subplot(subplot_count, i + 1)
        plt.imshow(output_images[i])
    return output_images

In [2]:
def max_pooling():
    pass

In [12]:
#filters
bottom_sobel =    [[-1,-2,-1],
                   [0, 0, 0],
                   [1, 2, 1]]
               
top_sobel =     [[1, 2, 1],
                 [0, 0, 0],
                 [-1,-2,-1]]

sharpening = [[0,-1,0],
              [-1,5,-1],
              [0,-1,0]]

weighted_averaging_3_3 = (1/16)*np.array([[1, 2, 1],
                                [2, 4, 2],
                                [1, 2, 1]])

robert_x = [[1, 0],
            [0, -1]]    

robert_y = [[0, +1],
            [-1, 0]] 

averaging_2x2 = (1/4)*np.array([[1, 1],
                       [1, 1]])

gaussian_5x5 = (1/273)*np.array([[1,4,7,4,1],
                        [4,16,26,16,4],
                        [7,26,41,26,7],
                        [4,16,26,16,4],
                        [1,4,7,4,1]])

averaging_5x5 = (1/25)*np.ones((5, 5))

first_layer_filters = [bottom_sobel, top_sobel, sharpening, weighted_averaging_3_3]
second_layer_filters = [robert_x, robert_y, averaging_2x2]
third_layer_filters = [gaussian_5x5, averaging_5x5]
